In [ ]:
import math
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as mpatches
import os
import glob
import numpy as np
from typing import Iterable
import pickle
import tensorflow as tf
from scipy.fft import fft, ifft
from statsmodels.tsa.stattools import acf, pacf
from sklearn.decomposition import PCA

import keras
import keras_tuner as kt
from kerastuner import RandomSearch
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras import regularizers

In [ ]:
# load a single shot using pandas parquet file reader 
def load_shot(shotnum: int) -> pd.DataFrame:
    shot = pd.read_parquet("../data/DISR_data/JET_DATAno" + str(shotnum) + ".parquet")
    return shot

df_shot = load_shot(96885)
df_shot['IPE'] = df_shot['IPLA'] - df_shot['IPref']
display(df_shot)

In [ ]:
model = tf.keras.models.load_model("../models/nn_window.keras")

window = 0

# Add 'Instability' and 'Window' attributes to the DataFrame
df_shot['Instability'] = 0
df_shot['Window'] = window
df_shot['Label'] = -1

# Define the window size and the number of samples with the same label required
window_size = 20
window_cursor_one = 0
window_cursor_two = window_size - 1
label_count = 15

# Iterate over the samples in the DataFrame
for i in range(len(df_shot)):
    # Get the current sample
    sample = df_shot.iloc[i]
    
    # Perform the prediction using the Keras model
    prediction = model.predict(np.array([sample]))[0]
    label = np.argmax(prediction)

    df_shot.loc[i, 'Label'] = label

    max_label = df_shot['Label'].iloc[window_cursor_one : window_cursor_two].value_counts().max()
    
    # Check if there are enough samples with the same label in the window
    if max_label >= label_count and max_label != -1:
        window += 1
        # Set the window attribute for the samples inside the window
        df_shot.loc[window_cursor_one : window_cursor_two, 'Window'] = window
        
        # Move the window to the next 20 samples
        window_cursor_one += window_size
        window_cursor_two += window_size

        # Check if the label is 1
        if label == 1:
            # Increase the instability by 1 for all following samples
            df_shot.loc[i + 1:, 'Instability'] += 1

    elif i == window_cursor_two:
        # Move the window to the next samples
        window_cursor_one += 1
        window_cursor_two += 1

# Display the updated DataFrame
display(df_shot)


In [ ]:
# Retrieve the unique window values different than 0 and where the label is equal to 1
window = df_shot[(df_shot['Window'] != 0) & (df_shot['Label'] == 1)]['Window'].unique()

# Print the dataframe window per window
display(df_shot[df_shot['Window'] == window[0]])